In [1]:
#import necessary library
import numpy as np # linear algebra
import pandas as pd # data manipulation and analysis
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
sns.set_style('whitegrid') # set style for visualization
import warnings # ignore warnings
warnings.filterwarnings('ignore')
from initial_report import *
import ast

In [2]:
#import dataset
df_events=pd.read_csv("events.csv")

In [3]:
#randomly get 5 rows
df_events.sample(5)

,customer_id,event,value,time
173270,aed59d1f47be4f73ba402ede49666b7a,offer viewed,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},420
901,cbecec5803434d63bce1e4c451d4b2d7,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
183129,1faf8fdf229c468ba054e98356c617b2,transaction,{'amount': 3.33},444
256867,ddc3173da9f645ddb3b1098994216fb5,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},576
224487,1e080ee5a39744dcb1f9d98efc23280f,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},516


In [4]:
#get_initial_report
initial_report(df_events)

 *** DATA CLEANING CHECKLIST ***
----------------------------------------
*** Structure:
- Total Rows: 306534
- Total Columns: 4
- Column Names: ['customer_id', 'event', 'value', 'time']

*** Data Types:
  customer_id: object
  event: object
  value: object
  time: int64

*** Mixed Data Types:

*** Distinct Values per Column:
  customer_id: 17000
  event: 4
  value: 5121
  time: 120

*** Null Values and Percentages:


*** Duplicates: 397

*** Negative or Zero Values:
  time: 15561

*** Basic Statistics:
                time
count  306534.000000
mean      366.382940
std       200.326314
min         0.000000
25%       186.000000
50%       408.000000
75%       528.000000
max       714.000000

*** Category Description:
                             customer_id        event  \
count                             306534       306534   
unique                             17000            4   
top     94de646f7b6041228ca7dec82adb97d2  transaction   
freq                                  51       

1. has duplicates
2. has negatives
3. need to check the data type of column-> value
4. check categroy description for event column

In [5]:
#get counts for column: event
df_events.event.value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [6]:
#see all the duplicates
df_events[df_events.duplicated(keep=False)]

,customer_id,event,value,time
66122,3dde94fa581145cb9f206624f1a94d5a,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,168
66123,3dde94fa581145cb9f206624f1a94d5a,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,168
66782,e9fb6ed2cecb4980ba98c86abc9c91e3,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,168
66783,e9fb6ed2cecb4980ba98c86abc9c91e3,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,168
67613,a7dc060f6fc94ca7bf71fbb188187dca,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,168
...,...,...,...,...
304756,0785f1fce0b04ba08e01c7d2ebab4917,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,708
305550,b7e216b6472b46648272c29a52a86702,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714
305551,b7e216b6472b46648272c29a52a86702,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714
306455,f3e801caeafe4899b3b989b586e74ac7,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,714


In [7]:
df_events.duplicated().sum()

np.int64(397)

In [8]:
#delete duplicates
df_events.drop_duplicates(inplace=True)

In [9]:
df_events.shape

(306137, 4)

In [10]:
#check data type for value
df_events['value'].apply(type).value_counts()

value
<class 'str'>    306137
Name: count, dtype: int64

In [11]:
#convert 'value" to dict
df_events['value'] = df_events['value'].apply(ast.literal_eval)

In [12]:
#again: check data type for value
df_events['value'].apply(type).value_counts()

value
<class 'dict'>    306137
Name: count, dtype: int64

In [13]:
#check list of keys
from itertools import chain

# Create a flat list of all keys used in the 'value' column
all_keys = list(set(chain.from_iterable(df_events['value'].apply(lambda x: x.keys()))))

print(all_keys)


['amount', 'offer_id', 'offer id', 'reward']


1.there are two offer_id; need to standardize

In [14]:
#standardize offer_id keys
def standardize_keys(d):
    return {k.replace(' ', '_').lower(): v for k, v in d.items()}

df_events['value'] = df_events['value'].apply(standardize_keys)


In [15]:
#check again the list
from itertools import chain

# Create a flat list of all keys used in the 'value' column
all_keys = list(set(chain.from_iterable(df_events['value'].apply(lambda x: x.keys()))))

print(all_keys)

['offer_id', 'reward', 'amount']


In [16]:
## Expand the dictionary into separate columns
value_expanded = df_events['value'].apply(pd.Series)

# Join back to the original DataFrame
df_events = df_events.drop('value', axis=1).join(value_expanded)

In [17]:
#check sample
df_events.sample(5)

,customer_id,event,time,offer_id,amount,reward
291854,4384b18804874776b628f5d6a390e2d4,transaction,654,NaN,11.85,NaN
287539,9e90ae7dd5e142d9ba59196ce702f0e3,offer completed,636,fafdcd668e3743c1bb461111dcafc2a4,NaN,2.0
251988,c5ecd92a7d1f442fb098df6d6ab8a3e5,offer received,576,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN
161990,1b71097607df424785b1c65f89c60aba,offer received,408,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN
29351,11bfa14bbbdc43a0a9007c7e71dd48ba,offer completed,42,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,5.0


In [18]:
#chheck negative in time column
df_events[df_events.time<0]

,customer_id,event,time,offer_id,amount,reward


1. No negative no problem

In [19]:
#save file
df_events.to_csv('cleaned_events.csv', index=False)

In [20]:
df_events.event.value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33182
Name: count, dtype: int64